In [1]:
!pip install -q transformers peft datasets -U bitsandbytes accelerate sentencepiece pillow tqdm huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.1/367.1 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import os
import json
import requests
from PIL import Image
from datasets import Dataset as HFDataset, Features, Image as HFImage, Value
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import LlavaForConditionalGeneration, LlavaProcessor, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from torch.optim import AdamW
from huggingface_hub import snapshot_download, hf_hub_download
import zipfile
import glob
import re

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cuda


In [ ]:
for split in ["train", "val"]:
    folder = "train_val"
    hf_hub_download(repo_id="google/spiqa", filename=f"{folder}/SPIQA_{split.replace('-', '')}.json", repo_type="dataset", local_dir="./spiqa_data")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


train_val/SPIQA_train.json:   0%|          | 0.00/209M [00:00<?, ?B/s]

SPIQA_val.json: 0.00B [00:00, ?B/s]

In [ ]:
def extract_reasoning_samples(local_dir="/content/spiqa_data", image_dir="/content/drive/MyDrive/SPIQA_train_val_Images", max_samples=500):
    # Path to metadata
    metadata_path = os.path.join(local_dir, "train_val", "SPIQA_train.json")

    # Download metadata if not present
    if not os.path.exists(metadata_path):
        os.makedirs(os.path.dirname(metadata_path), exist_ok=True)
        try:
            hf_hub_download(repo_id="google/spiqa", filename="train_val/SPIQA_train.json", repo_type="dataset", local_dir=local_dir)
            print(f"Downloaded SPIQA_train.json to {metadata_path}")
        except Exception as e:
            print(f"Error downloading train metadata: {e}")
            return None

    with open(metadata_path, 'r') as f:
        metadata = json.load(f)

    print(f"Loaded metadata for train with {len(metadata)} papers")
    if len(metadata) > 0:
        sample_paper = list(metadata.items())[0]
        print(f"Sample paper structure: {sample_paper[0]} -> {sample_paper[1].keys()}")
        if 'qa' in sample_paper[1]:
            print(f"Sample QA pair: {sample_paper[1]['qa'][0] if sample_paper[1]['qa'] else 'No QA pairs'}")

    # Collect all images from Drive
    image_paths = glob.glob(os.path.join(image_dir, "*", "*.png"))
    print(f"Found {len(image_paths)} images in {image_dir}")

    samples = []
    unmatched_images = set()
    used_images = set()  # Track used images to avoid duplicates
    used_qa_indices = {}  # Track used QA pairs per paper to avoid duplicates

    for image_path in tqdm(image_paths, desc="Processing images"):
        if len(samples) >= max_samples:
            print(f"Reached maximum of {max_samples} samples, stopping")
            break

        # Extract paper_id and figure_id from image path
        paper_id = os.path.basename(os.path.dirname(image_path))
        image_name = os.path.basename(image_path)
        # Match figure or table identifier (e.g., Figure1-1, Table1-1)
        figure_match = re.search(r'(?:Figure|Fig|Table|table)(\d+)(?:-1)?', image_name, re.IGNORECASE)
        if not figure_match:
            print(f"Warning: Could not parse figure or table ID from {image_name}, skipping")
            unmatched_images.add(image_path)
            continue
        figure_id = f"{figure_match.group(1).title()}{figure_match.group(1)}"  # Normalize to FigureX or TableX

        # Find matching QA pair in metadata
        qa_list = metadata.get(paper_id, {}).get('qa', [])
        if not qa_list:
            print(f"Warning: No QA pairs for paper {paper_id}, skipping image {image_name}")
            unmatched_images.add(image_path)
            continue

        matched_qa = None
        qa_index = None
        # Try matching with image_id first
        for i, qa in enumerate(qa_list):
            image_id = qa.get('image_id', paper_id)
            if image_id in image_name or f"{image_id}-1" in image_name:
                if paper_id not in used_qa_indices or i not in used_qa_indices.get(paper_id, set()):
                    matched_qa = qa
                    qa_index = i
                    break

        # Fallback to description matching
        if matched_qa is None:
            for i, qa in enumerate(qa_list):
                description = qa.get('description', '')
                if figure_id.lower() in description.lower() or f"{figure_id}-1".lower() in description.lower():
                    if paper_id not in used_qa_indices or i not in used_qa_indices.get(paper_id, set()):
                        matched_qa = qa
                        qa_index = i
                        break

        # Fallback to index-based matching (first unmatched image to first unmatched QA pair)
        if matched_qa is None:
            for i, qa in enumerate(qa_list):
                if paper_id not in used_qa_indices or i not in used_qa_indices.get(paper_id, set()):
                    matched_qa = qa
                    qa_index = i
                    break

        if matched_qa is None:
            print(f"Warning: No QA pair matched for {image_name} in paper {paper_id}, skipping")
            unmatched_images.add(image_path)
            continue

        # Load image and create sample
        try:
            image_data = Image.open(image_path).convert("RGB")
        except Exception as e:
            print(f"Error opening image {image_path}: {e}")
            unmatched_images.add(image_path)
            continue

        question = matched_qa.get('question', '')
        answer = matched_qa.get('answer', '')
        image_id = matched_qa.get('image_id', paper_id)
        if question and answer and image_path not in used_images:
            samples.append({
                "image": image_data,
                "image_id": image_id,
                "image_path": image_path,
                "question": question,
                "answer": answer,
                "split": "train"
            })
            used_images.add(image_path)
            used_qa_indices.setdefault(paper_id, set()).add(qa_index)
            print(f"Added sample: image_id={image_id}, question={question[:50]}...")
            print(f"Matched via: {'image_id' if image_id in image_name or f'{image_id}-1' in image_name else 'description' if figure_id.lower() in matched_qa.get('description', '').lower() else 'index-based'}")

    print(f"Extracted {len(samples)} samples from train")
    if unmatched_images:
        print(f"Unmatched images (sample): {list(unmatched_images)[:10]}")
    if not samples:
        print("Warning: No samples extracted. Check image availability or metadata matching.")
        return None

    dataset = HFDataset.from_dict({
        "image": [s["image"] for s in samples],
        "image_id": [s["image_id"] for s in samples],
        "image_path": [s["image_path"] for s in samples],
        "question": [s["question"] for s in samples],
        "answer": [s["answer"] for s in samples],
        "split": [s["split"] for s in samples]
    }, features=Features({
        "image": HFImage(),
        "image_id": Value("string"),
        "image_path": Value("string"),
        "question": Value("string"),
        "answer": Value("string"),
        "split": Value("string")
    }))

    print(f"Created dataset with {len(dataset)} samples")
    return dataset

def save_preprocessed_data(output_file="/content/data/spiqa_train_subset.jsonl"):
    """
    Save preprocessed train samples to local Colab storage, copying images to /content/images/.

    Args:
        output_file (str): Output file for dataset.
    """
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    os.makedirs("/content/images", exist_ok=True)

    dataset = extract_reasoning_samples()
    if dataset is None:
        raise ValueError("No samples extracted. Check dataset structure or image availability.")

    valid_samples = 0
    with open(output_file, "w") as f:
        for s in tqdm(dataset, desc="Saving preprocessed data"):
            try:
                # Copy image to /content/images/ for training efficiency
                new_image_path = f"/content/images/{os.path.basename(s['image_path'])}"
                if s["image"] is not None:
                    s["image"].save(new_image_path)
                    print(f"Copied image to {new_image_path}")
                    record = {
                        "image_path": new_image_path,
                        "question": s["question"],
                        "answer": s["answer"],
                        "split": s["split"]
                    }
                    f.write(json.dumps(record) + "\n")
                    valid_samples += 1
                else:
                    print(f"Skipping sample {s['image_id']} (no image data)")
            except Exception as e:
                print(f"Error saving sample {s['image_id']}: {e}")
    print(f"Saved {valid_samples} valid samples to {output_file}")
# Mount Google Drive (for image access only)
from google.colab import drive
drive.mount('/content/drive')

# Check disk space
!df -h

# Preprocess train samples (up to 5,000)
save_preprocessed_data(output_file="/content/data/spiqa_train_subset.jsonl")

Mounted at /content/drive
Filesystem      Size  Used Avail Use% Mounted on
overlay         113G   43G   71G  38% /
tmpfs            64M     0   64M   0% /dev
shm             5.7G  4.0K  5.7G   1% /dev/shm
/dev/root       2.0G  1.2G  775M  61% /usr/sbin/docker-init
tmpfs           6.4G  660K  6.4G   1% /var/colab
/dev/sda1        74G   47G   27G  64% /kaggle/input
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware
drive           113G   47G   67G  41% /content/drive
Loaded metadata for train with 25459 papers
Sample paper structure: 1612.01810v3 -> dict_keys(['paper_id', 'all_figures', 'qa'])
Sample QA pair: {'question': 'Which method converges faster, joint or separate assignment and update steps?', 'answer': 'The joint method converges faster than the separate method.', 'explanation': 'The figure shows the boundary recall of the two methods over the number of iterations. The joint method

Processing images:   0%|          | 1/24398 [00:00<6:35:47,  1.03it/s]

Added sample: image_id=2010.02498v1, question=What are the four dimensions of linguistic quality...
Matched via: image_id


Processing images:   0%|          | 2/24398 [00:01<6:06:52,  1.11it/s]

Added sample: image_id=2010.02498v1, question=Which metric has the highest correlation with huma...
Matched via: image_id


...

Copied image to /content/images/2008.11783v1-Figure7-1.png
Copied image to /content/images/2008.11783v1-Figure6-1.png
Copied image to /content/images/2008.11783v1-Figure2-1.png
Copied image to /content/images/2008.11783v1-Table7-1.png
Copied image to /content/images/2008.11783v1-Figure1-1.png
Copied image to /content/images/2008.11783v1-Table5-1.png
Copied image to /content/images/2008.11783v1-Table1-1.png
Copied image to /content/images/2008.09514v1-Figure2-1.png
Copied image to /content/images/2008.09514v1-Table5-1.png
Copied image to /content/images/2008.09514v1-Table2-1.png
Copied image to /content/images/2008.09514v1-Figure4-1.png


Saving preprocessed data:  99%|█████████▉| 494/500 [00:12<00:00, 45.46it/s]

Copied image to /content/images/2008.09514v1-Table4-1.png
Copied image to /content/images/2008.09514v1-Figure1-1.png
Copied image to /content/images/2008.09514v1-Table1-1.png
Copied image to /content/images/2008.09514v1-Table3-1.png
Copied image to /content/images/2008.09514v1-Figure5-1.png
Copied image to /content/images/2008.11087v2-Table3-1.png
Copied image to /content/images/2008.11087v2-Figure3-1.png
Copied image to /content/images/2008.11087v2-Figure1-1.png
Copied image to /content/images/2008.11087v2-Figure5-1.png
Copied image to /content/images/2008.11087v2-Table5-1.png
Copied image to /content/images/2008.11087v2-Figure4-1.png


Saving preprocessed data: 100%|██████████| 500/500 [00:12<00:00, 38.68it/s]

Copied image to /content/images/2008.11087v2-Figure6-1.png
Copied image to /content/images/2008.11087v2-Table2-1.png
Copied image to /content/images/2008.11087v2-Figure2-1.png
Copied image to /content/images/2008.11087v2-Table4-1.png
Saved 500 valid samples to /content/data/spiqa_train_subset.jsonl


In [ ]:
# Custom Dataset for SPIQA with LLaVA processor
class SpiqaReasoningDataset(Dataset):
    def __init__(self, data_path, processor):
        self.processor = processor
        with open(data_path) as f:
            self.samples = [json.loads(line) for line in f]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        try:
            image = Image.open(sample["image_path"]).convert("RGB")
        except FileNotFoundError:
            print(f"Image not found: {sample['image_path']}, using blank image")
            image = Image.new("RGB", (224, 224), (255, 255, 255))  # Blank white image
        prompt = f"USER: <image>\n{sample['question']}\nASSISTANT: {sample['answer']}"
        return image, prompt

# Collate function for batching
def collate_fn(batch):
    images, prompts = zip(*batch)
    inputs = processor(
        text=list(prompts),
        images=list(images),
        return_tensors="pt",
        padding=True,
    )
    inputs["labels"] = inputs["input_ids"].clone()
    return inputs

In [ ]:
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

model_id = "llava-hf/llava-1.5-7b-hf"
processor = LlavaProcessor.from_pretrained(model_id)

# Updated quantization config with CPU offload enabled
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    llm_int8_enable_fp32_cpu_offload=True  # This is the key addition
)

model = LlavaForConditionalGeneration.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    quantization_config=quant_config,
)

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

preprocessor_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [ ]:
model = get_peft_model(model, lora_config)
model.to(device)
model.train()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlavaForConditionalGeneration(
      (model): LlavaModel(
        (vision_tower): CLIPVisionModel(
          (vision_model): CLIPVisionTransformer(
            (embeddings): CLIPVisionEmbeddings(
              (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
              (position_embedding): Embedding(577, 1024)
            )
            (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (encoder): CLIPEncoder(
              (layers): ModuleList(
                (0-23): 24 x CLIPEncoderLayer(
                  (self_attn): CLIPAttention(
                    (k_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
                    (v_proj): lora.Linear4bit(
                      (base_layer): Linear4bit(in_features=1024, out_features=1024, bias=True)
                      (lora_dropout): ModuleDict(
                        (default): Dropout

In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-5)

# %%
# 7. Create DataLoader with batching
dataset = SpiqaReasoningDataset("data/spiqa_train_subset.jsonl", processor)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

# %%
# 8. Training loop with batching
epochs = 1
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    for step, batch in enumerate(tqdm(dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if (step + 1) % 10 == 0:
            print(f"Step {step+1}, Loss: {loss.item():.4f}")

# %%
# 9. Save the fine-tuned LoRA adapter
model.save_pretrained("/content/drive/MyDrive/model/llava-lora-reasoning")
print("Saved LoRA adapter to model/llava-lora-reasoning")

Epoch 1/1


  0%|          | 0/63 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
 16%|█▌        | 10/63 [10:10<55:13, 62.52s/it]

Step 10, Loss: 11.1357


 32%|███▏      | 20/63 [20:57<47:51, 66.79s/it]

Step 20, Loss: 8.6655


...

Step 50, Loss: 5.0120


 95%|█████████▌| 60/63 [1:02:45<03:13, 64.45s/it]

Step 60, Loss: 4.7438


100%|██████████| 63/63 [1:05:19<00:00, 62.21s/it]


Saved LoRA adapter to model/llava-lora-reasoning


In [6]:
def answer_question(image, question, processor, model, device, max_tokens=100):
    """
    Generate an answer for a given image and question using the fine-tuned LLaVA model.

    Args:
        image (PIL.Image.Image): Input image as a PIL Image object.
        question (str): Question to answer based on the image.
        processor: LLaVA processor for tokenizing inputs.
        model: Fine-tuned LLaVA model.
        device: Device to run the model on (e.g., 'cuda' or 'cpu').
        max_tokens (int): Maximum number of tokens to generate.

    Returns:
        str: Generated answer text.
    """
    try:
        # Ensure image is in RGB format
        image = image.convert("RGB")
        prompt = f"USER: <image>\n{question}\nASSISTANT:"
        inputs = processor(text=prompt, images=image, return_tensors="pt").to(device)
        output_ids = model.generate(**inputs, max_new_tokens=max_tokens)
        output_text = processor.batch_decode(output_ids, skip_special_tokens=True)[0]
        # Extract the answer part after "ASSISTANT:"
        answer = output_text.split("ASSISTANT:")[-1].strip()
        return answer
    except Exception as e:
        print(f"Error generating answer: {e}")
        return "Error: Could not generate an answer."


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
from io import BytesIO
from peft import PeftModel, PeftConfig


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = LlavaProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)
base_model = LlavaForConditionalGeneration.from_pretrained(
    "llava-hf/llava-1.5-7b-hf",
    device_map="auto",
    torch_dtype=torch.float16,
    quantization_config=quant_config,
)
# Load the LoRA adapter
model = PeftModel.from_pretrained(
    base_model,
    "/content/drive/MyDrive/llava-lora-reasoning",  # Updated path
    torch_dtype=torch.float16,
)

model.eval()
print("Model and processor loaded successfully")

# Create interactive widgets
question_input = widgets.Text(
    value="",
    placeholder="Enter your question (e.g., What is the main algorithm described in the figure?)",
    description="Question:",
    layout={'width': '80%'}
)
image_upload = widgets.FileUpload(
    accept=".png,.jpg,.jpeg",
    multiple=False,
    description="Upload Image"
)
output = widgets.Output()
button = widgets.Button(description="Get Answer")

model.eval()

# Create interactive widgets
question_input = widgets.Text(
    value="",
    placeholder="Enter your question (e.g., What is the main algorithm described in the figure?)",
    description="Question:",
    layout={'width': '80%'}
)
image_upload = widgets.FileUpload(
    accept=".png,.jpg,.jpeg",
    multiple=False,
    description="Upload Image"
)
output = widgets.Output()
button = widgets.Button(description="Get Answer")

def on_button_clicked(b):
    with output:
        clear_output()
        question = question_input.value.strip()
        if not question:
            print("Please enter a question.")
            return

        # Get image from upload
        if image_upload.value:
            uploaded_file = list(image_upload.value.values())[0]
            image_data = uploaded_file["content"]
            try:
                image = Image.open(BytesIO(image_data)).convert("RGB")
            except Exception as e:
                print(f"Error loading image: {e}, using blank image")
                image = Image.new("RGB", (224, 224), (255, 255, 255))
        else:
            print("No image uploaded, using blank image")
            image = Image.new("RGB", (224, 224), (255, 255, 255))

        # Generate answer
        answer = answer_question(image, question, processor, model, device, max_tokens=100)
        print(f"Question: {question}\nAnswer: {answer}")

button.on_click(on_button_clicked)

# Display widgets
display(question_input, image_upload, button, output)